# Solar notebook example

This notebook is a simple example to demonstrate how to use the `solar_maintenance_app` to monitor the energy production of a PV system. Follow the steps below to set up and run the notebook.

---

### Step 1: Configure Environment Variables

To execute this notebook successfully, ensure the following environment variables are defined. These can be stored in a `.env` file in the repository root:

- `WEATHER_SERVICE_ADDRESS`: The address of the weather service. For more information please refer to [frequenz-api-weather](https://github.com/frequenz-floss/frequenz-api-weather).
- `REPORTING_SERVICE_ADDRESS`: The address of the reporting service. For more information please refer to [frequenz-client-reporting-python](https://github.com/frequenz-floss/frequenz-client-reporting-python).
- `REPORTING_API_KEY`: The API key for the reporting service.

**Example `.env` file:**
```plaintext
WEATHER_SERVICE_ADDRESS=http://weatherapi.example.com
REPORTING_SERVICE_ADDRESS=http://reportingapi.example.com
REPORTING_API_KEY=your-api-key

### Step 2: Define `all_sites`

The `all_sites` list must be configured with the details of your microgrid sites. Each entry in `all_sites` should be an instance of the `SiteConfig` dataclass.

Example:
```
all_sites = [
    SiteConfig(
        mid=0,
        component_ids=[0, 1],
        client_site_info=ClientSiteInfo(
            latitude=0.0,  # Geographic latitude
            longitude=0.0,  # Geographic longitude
            peak_power_watts=0.0,  # Peak power of solar modules (in watts)
            rated_power_watts=0.0,  # Rated power of PV inverters (in watts)
            efficiency=0.9  # (Optional) System efficiency (default=0.85)
        )
    ),
]
```

Refer to the code cell with the `ClientSiteInfo` and `SiteConfig` dataclasses for more details.

### Step 3: Update `USER_REQUEST`

Adjust the `USER_REQUEST` dictionary to customise the parameters for your analysis. Key fields include:
- `start_timestamp` and `end_timestamp`: Define the analysis time range.
- `rolling_view_duration`: Set the duration (number of days) for the long-term view plots.
- `large_resample_period_seconds`: Resample period (seconds) for processing/displaying data with low temporal resolution. Used for all plots except "real_time_view".
- `small_resample_period_seconds`: Resample period (seconds) for processing/displaying data with high temporal resolution. Used only for "real_time_view".
- `baseline_models`: Specify the baseline models for comparison.
- `time_zone`: Set the timezone for the analysis (e.g., "Europe/Berlin"). This only affects the displayed time axes.


### Additional Notes

Reference `config.py`: For more details on configurable parameters, check the `config.py` file in the project's directory.

---
Once the setup is complete, run the notebook to view the data. Adjust configurations as needed for your specific use case.

In [ ]:
import datetime
import logging
import os
from dataclasses import asdict, dataclass, field

from dotenv import load_dotenv

from frequenz.lib.notebooks.solar.maintenance.solar_maintenance_app import run_workflow

In [ ]:
logging.basicConfig(
    level=logging.WARNING,
    format="%(asctime)s %(levelname)s %(name)s: %(message)s",
    datefmt="%H:%M:%S",
)
logging.getLogger("matplotlib.font_manager").setLevel(logging.ERROR)
logging.getLogger("frequenz.lib.notebooks").setLevel(logging.WARNING)

In [ ]:
# Load environment variables from the .env file
load_dotenv()

In [ ]:
"""Dataclass for the configuration of the client site."""

@dataclass
class ClientSiteInfo:
    """Detailed configuration for specific client site information."""

    latitude: float = field(
        metadata={
            "description": "Geographic latitude of the site.",
            "required": True,
        },
    )

    longitude: float = field(
        metadata={
            "description": "Geographic longitude of the site.",
            "required": True,
        },
    )

    peak_power_watts: float = field(
        metadata={
            "description": "The peak power (solar panels) in Watts.",
            "required": True,
        },
    )

    rated_power_watts: float = field(
        metadata={
            "description": "The rated power (PV inverters) in Watts.",
            "required": True,
        },
    )

    efficiency: float = field(
        default=0.85,
        metadata={
            "description": "An estimate of the overall system efficiency.",
            "required": False,
            "validate": lambda x: 0.0 <= x <= 1.0,
        },
    )

    altitude: float = field(
        default=0.0,
        metadata={
            "description": "Geographic altitude of the site.",
            "required": False,  # only used for the pvlib simulation
        },
    )


@dataclass
class SiteConfig:
    """Main configuration for a client site entry, including component and site details."""

    mid: int = field(
        metadata={
            "description": "The microgrid ID of the client site",
            "required": True,
        },
    )

    component_ids: list[int] = field(
        metadata={
            "description": "List of component IDs associated with the site",
            "required": True,
        },
    )

    client_site_info: ClientSiteInfo = field(
        metadata={
            "description": "Detailed configuration for client site information",
            "required": True,
        },
    )


# define all sites here
all_sites = [
    # example usage:
    # asdict(
    #     SiteConfig(
    #         mid=0,
    #         component_ids=[0, 1],
    #         client_site_info=ClientSiteInfo(
    #             latitude=0.0,
    #             longitude=0.0,
    #             peak_power_watts=0.0,
    #             rated_power_watts=0.0,
    #             efficiency=0.0,
    #         ),
    #     )
    # ),
]

In [ ]:
# example user request (1 year of data)
end_time = datetime.datetime.now().astimezone(datetime.timezone.utc)
start_time = end_time - datetime.timedelta(days=14)

USER_REQUEST = {
    "weather_service_address": os.environ.get("WEATHER_SERVICE_ADDRESS"),
    "reporting_service_address": os.environ.get("REPORTING_SERVICE_ADDRESS"),
    "microgrid_ids": [x["mid"] for x in all_sites],
    "component_ids": [x["component_ids"] for x in all_sites],
    "client_site_info": [x["client_site_info"] for x in all_sites],
    "time_zone": "Europe/Berlin",
    "end_timestamp": end_time,
    "start_timestamp": start_time,
    "rolling_view_duration": 30,  # the time frame defaults to days
    "baseline_models": [
        "7-day MA",
        "7-day sampled MA",
        "weather-based-forecast",
    ],
    "large_resample_period_seconds": 3600,
    "small_resample_period_seconds": 2,
    "language": "en",
}

In [ ]:
if len(all_sites) > 0:
    plot_data = await run_workflow(user_config_changes=USER_REQUEST)